In [5]:
import io
from googleapiclient.discovery import build
from google.oauth2 import service_account
from PIL import Image
import requests
import base64

# Path to API key and service account file
api_key_file_path = '/Users/angelokaram/Desktop/Sac State Folders/Sac State Spring 2024/CSC 190/Senior-Project/private_keys.txt'
SERVICE_ACCOUNT_FILE = '/Users/angelokaram/Desktop/Sac State Folders/Sac State Spring 2024/CSC 190/Senior-Project/test-ai-sheet-1cbe18b0629d.json'

# Authentication and API setup
def get_api_key_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            if len(lines) >= 2:
                return lines[0].strip()  # API key on first line
            else:
                raise ValueError("API key not found in the file.")
    except Exception as e:
        print(f"Failed to read the API key from file: {e}")
        return None

# Set up the API key and credentials
API_KEY = get_api_key_from_file(api_key_file_path)
SCOPES = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
drive_service = build('drive', 'v3', credentials=credentials)

# list files in a folder by folder ID
def list_files_in_folder(folder_id):
    results = drive_service.files().list(
        q=f"'{folder_id}' in parents and mimeType != 'application/vnd.google-apps.folder'",
        fields="nextPageToken, files(id, name, mimeType)").execute()
    return results.get('files', [])

#  get subfolders in a folder
def list_folders_in_folder(folder_id):
    results = drive_service.files().list(
        q=f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.folder'",
        fields="nextPageToken, files(id, name)").execute()
    return results.get('files', [])

# download a file from Google Drive
def download_file(file_id, file_name):
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
    fh.seek(0)
    with open(file_name, 'wb') as f:
        f.write(fh.read())

# encode image for Google Vision API
def encode_image(image_path):
    with open(image_path, 'rb') as image_file:
        return base64.b64encode(image_file.read()).decode()

# detect text using Google Vision API
def detect_text_via_api(image_content):
    url = f"https://vision.googleapis.com/v1/images:annotate?key={API_KEY}"
    headers = {'Content-Type': 'application/json'}
    
    body = {
        "requests": [
            {
                "image": {
                    "content": image_content
                },
                "features": [
                    {"type": "DOCUMENT_TEXT_DETECTION"}  # for handwritten text
                ]
            }
        ]
    }

    # API request and response handling
    response = requests.post(url, headers=headers, json=body)
    result = response.json()
    print(result)  # Debugging: prints the API response

    # Process the response to identify symbols and text
    if 'responses' in result and len(result['responses']) > 0 and 'textAnnotations' in result['responses'][0]:
        annotations = result['responses'][0]['textAnnotations']
        for annotation in annotations:
            description = annotation['description']
            print(description)  # Print out each detected text block to validate
        return annotations[0]['description']
    else:
        return 'No text detected'

# analyze images using Google Vision API
def analyze_images(image_files):
    for image_file in image_files:
        image_content = encode_image(image_file)
        result = detect_text_via_api(image_content)
        print(result)

# Example usage: TODO -> Replace with your Submissions folder ID, or the client's
submissions_folder_id = '1aAEJ5YXy09YJKr1p359abvJx-ltOrZ5h'
